In [ ]:
import torchaudio
import IPython.display as ipd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt

from model import *
from dataset import *
from const import *
from train import *

In [ ]:
from singlefit import *
def single_inference():
    net = SingleDecoder()
    net.load_state_dict(torch.load('data/ckpts/single_latest.sd'))
    ds, loader = get_lj_loader(batch_size=1, limit=1, get_dataset=True)
    net.to(device)
    with torch.no_grad():
        for batch in loader:
            S_pad, S_lengths, token_pad, token_lengths = batch
            S_pad = S_pad.to(device)
            S_pred = net(S_pad)

    wav, _, _, _ = ds.base_ds[0]
    
    S_base_ds = ds.wav_to_spec(wav)
    S_ds, _ = ds[0]
    S_loaded = S_pad[:, 0, :].cpu()
    
    #recon_wav = ds.spec_to_wav(S_base_ds)
    #recon_wav = ds.spec_to_wav(S_ds)
    recon_wav = ds.spec_to_wav(S_loaded)
    sr = 22050
    librosa.output.write_wav('data/recon_sample.wav', recon_wav.squeeze().numpy(), sr)
    
single_inference()

In [ ]:
def torch_reconstruct_demo():
    sr = 22050
    ds = LinSpecDataset(torchaudio.datasets.LJSPEECH('./data'), limit=1)
    orig_wav, _, _, _ = ds.base_ds[0]
    orig_S = ds.wav_to_spec(orig_wav)
    print(orig_S.shape)
    recon_wav = ds.spec_to_wav(orig_S)
    librosa.output.write_wav('data/recon_sample.wav', recon_wav.squeeze().numpy(), sr)
torch_reconstruct_demo()

In [ ]:
ipd.Audio('data/orig_sample.wav')

In [ ]:
ipd.Audio('data/recon_sample.wav')